# A/B Email Campaign Testing Project
## Data Cleaning

Import Libraries

In [4]:
import numpy as np
import pandas as pd

Loading Dataset

In [7]:
# Load the dataset and parse dates in proper format
df = pd.read_csv('../data/raw/email_campaign_data.csv', parse_dates=['timestamp'])

View Data

In [8]:
df.head()

,user_id,variant,send_time,content_layout,account_age,feature_usage,user_segment,open,click,convert,timestamp,device
0,1,A,afternoon,text-heavy,216.0,48.0,active,1.0,1.0,0.0,2023-04-01 16:04:01,desktop
1,2,B,morning,text-heavy,1.0,20.0,new,1.0,1.0,0.0,2023-04-14 09:26:41,desktop
2,3,A,morning,balanced,357.0,47.0,active,0.0,0.0,0.0,2023-04-14 08:32:52,desktop
3,4,A,evening,balanced,262.0,33.0,active,1.0,1.0,0.0,2023-04-03 20:44:07,mobile
4,5,A,morning,visual-heavy,73.0,8.0,active,0.0,0.0,0.0,2023-04-12 11:52:06,desktop


In [11]:
print("Rows, Columns")
df.shape

Rows, Columns


(515000, 12)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515000 entries, 0 to 514999
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   user_id         515000 non-null  int64         
 1   variant         515000 non-null  object        
 2   send_time       515000 non-null  object        
 3   content_layout  515000 non-null  object        
 4   account_age     504698 non-null  float64       
 5   feature_usage   509837 non-null  float64       
 6   user_segment    512443 non-null  object        
 7   open            513681 non-null  float64       
 8   click           515000 non-null  float64       
 9   convert         515000 non-null  float64       
 10  timestamp       515000 non-null  datetime64[ns]
 11  device          515000 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(5)
memory usage: 47.1+ MB


Handling Missing Data

In [13]:
df.isnull().sum()

user_id               0
variant               0
send_time             0
content_layout        0
account_age       10302
feature_usage      5163
user_segment       2557
open               1319
click                 0
convert               0
timestamp             0
device                0
dtype: int64

In [14]:
# Handle missing data for account_age and feature_usage using median value
df['account_age'] = df['account_age'].fillna(df['account_age'].median())
df['feature_usage'] = df['feature_usage'].fillna(df['feature_usage'].median())

In [15]:
# for user_segment we will replace with unknown
df['user_segment'] = df['user_segment'].fillna('unknown')

In [16]:
# For email open we will assume non-tracked as non-opened (replace with 0)
df['open'] = df['open'].fillna(0)

In [17]:
# Recheck null values
df.isnull().sum()

user_id           0
variant           0
send_time         0
content_layout    0
account_age       0
feature_usage     0
user_segment      0
open              0
click             0
convert           0
timestamp         0
device            0
dtype: int64

Removing duplicates if available

In [18]:
df.duplicated(subset=['user_id', 'timestamp']).sum()

np.int64(15000)

In [19]:
# Remove 15000 duplicates
df = df.drop_duplicates(subset=['user_id', 'timestamp'], keep='first')

df.duplicated(subset=['user_id', 'timestamp']).sum()

np.int64(0)

Data Validation

In [21]:
print("Data types before:\n", df.dtypes)
df['user_id'] = df['user_id'].astype(int)
df['variant'] = df['variant'].astype(str)
df['send_time'] = df['send_time'].astype(str)
df['content_layout'] = df['content_layout'].astype(str)
df['user_segment'] = df['user_segment'].astype(str)
df['device'] = df['device'].astype(str)
df['open'] = df['open'].astype(int)
df['click'] = df['click'].astype(int)
df['convert'] = df['convert'].astype(int)

print("Date types after:\n", df.dtypes)

Data types before:
 user_id                    int64
variant                   object
send_time                 object
content_layout            object
account_age              float64
feature_usage            float64
user_segment              object
open                       int64
click                      int64
convert                    int64
timestamp         datetime64[ns]
device                    object
dtype: object
Date types after:
 user_id                    int64
variant                   object
send_time                 object
content_layout            object
account_age              float64
feature_usage            float64
user_segment              object
open                       int64
click                      int64
convert                    int64
timestamp         datetime64[ns]
device                    object
dtype: object


Handling Invalid Timestamps

In [23]:
df['timestamp'].isna().sum()

# if there is invalid timestamps we will handle them as follows (uncomment)
# invalid_timestamps = df['timestamp'].isna().sum()

# if invalid_timestamps > 0:
#     print(f'Found {invalid_timestamps} invalid timestamps. Filling with median timestamp...')
#     df['timestamp'] = df['timestamp'].fillna(df['timestamp'].median())

np.int64(0)

Saving Cleaned CSV for Analysis

In [24]:
df.to_csv('../data/processed/cleaned_email_data.csv', index=False)